## Developer Test 

The below is a series of responses to a developer test. 

I've picked a jupyter notebook to help work these as a series of "mini challenges" as opposed to having to worry about several different environments etc. 

***

# Q1 - Find a solution for the problem: 

*Given a sentence and a maximum length, break the sentence down in chunks not longer than the maximum length, without breaking words in the middle. Check if its possible to find a solution that doesn't read the sentence more than once.*

---

Notes/Plan: 

* Seems to require a function that allows for "chunking" of a sentence? 
    * A sentence in python can be treated like a list, hence the concern about breaking words in the middle? 
* The best course of action is to create a function which splits a sentence on whitespace, then iterates through chunking it once. 
    * Does this count as reading the sentence more than once? 


In [ ]:
## Solution to problem
def sentence_chunker(sentence, max_length):

    sentence_parts = sentence.split(" ")
    chunks = [sentence_parts[i:i+max_length] for i in range(0, len(sentence_parts), max_length)]
    
    print (f"sentence: {sentence}\nmaximum chunk length: {max_length}\nevidence: {chunks}")
    
    return chunks

## Test to assert that solution works
def chunk_test(chunks, max_length): 
    
    print ("Beginning testing...")
    
    for chunk in chunks: 
        
        assert(len(chunk) <= max_length), f"The chunk: {chunk} is longer than the max length of {max_length}"
        
    print ("Testing successful!")

## Simple Script Usage

The below is a simple script usage of the two functions above, one to solve the problem, and then one to prove the results pass the criteria. 

It provides a sentence, and a max length, and passes them through to the sentence breaker function. 

The print statements are set up to just look nice in a jupyter notebook environment.

In [ ]:
chunks = sentence_chunker(sentence = "The quick brown fox jumped over the lazy dog", 
                          max_length = 4)


## Assert that it meets the problem criteria
chunk_test(chunks, max_length = 4 )

***

# Q2 - Enhance the solution: 

*Enhance the solution to Q1 so that it can read the arguments and write the results into a NoSQL database of your choice.*

***

Notes/Plan: 

* My NoSQL database of choice is [TinyDB](https://tinydb.readthedocs.io/en/latest/index.html). It's a tiny document oriented database written in pure Python and has no external dependencies. I particularly like it as it's able to provide a json file out the other end, and no network hosting / db things have to be considered for simple examples. I've also used it in the past to be the backend to an extensive time-resourcing system I designed using Excel as a front end, and had no ability to host a live database on the network. 
* An additional benefit of this is that I don't have to worry about driver installations etc.

In [12]:
from tinydb import TinyDB

class SentenceChunker:    
    
    """A class handling sentence chunking
    
    Attributes
    ----------
    sentence: str
        The sentence you wish to break up into specific sized chunks
    max_length: int
        The maximum length you wish your sentence chunks to be
    db_target_name: str
        Not compulsory, but if specifed, is the path to the json db you wish to store results
    
    Methods
    -------
    chunk():
        Break provided sentence into chunks, size dictated by max length
    chunk_test():
        Assert that the sentence has been correctly chunked, and does not exceed max_length
    store_results(): 
        Store chunk results within tinydb json database
    chunk_and_store(): 
        On request, chunk and store results in tinydb json database in user location of choice
    """
    
    
    def __init__(self, sentence:str, max_length:int, db_target_name:str = None): 
        
        self.sentence = sentence
        self.max_length = max_length 
        self.db_target_name = db_target_name
    
    def chunk(self): 
        
        """Break the provided sentence into chunks, size dictated by max length
        
        Returns
        -------
        list of lists
        """
        
        sentence_parts = self.sentence.split(" ")
        chunks = [sentence_parts[i:i+self.max_length] for i in range(0, len(sentence_parts), self.max_length)]
        
        print (f"Converted --> {self.sentence}\ninto --> {chunks}")
        
        self.chunk_test(chunks, self.max_length)
        
        return chunks
    
    @staticmethod
    def chunk_test(chunks, max_length): 

        """Assert that the sentence has been broken into chunks no greater than max length
        
        Returns
        -------
        None
        """
        
        for chunk in chunks: 

            assert(len(chunk) <= max_length), f"The chunk: {chunk} is longer than the max length of {max_length}"
            
    def store_results(self, chunks): 
        
        """Store chunk results within tinydb json database
        
        Returns
        -------
        None
        """
        
        db = TinyDB(self.db_target_name)
        db.insert({"sentence":self.sentence, "max_length":self.max_length, "chunk_results":chunks})
        print ("Inserted into database!")
        
    def chunk_and_store(self): 
        
        """On request, chunk and store results in tinydb json database in user location of choice
        
        Returns
        ------
        None"""
        
        ## Break sentence up
        chunks = self.chunk()
        
        ## Test it was broken successfully
        self.chunk_test(chunks, self.max_length)
        
        ## Store results in json db
        self.store_results(chunks)    
        

## Use Cases

The below example is to show how you would use the class to still chunk the results -- the test feature to ensure it's correctly chunking to sizes no greater than the max length is embedded within the chunk function.

In [13]:
SentenceChunker(sentence = "The quick brown fox jumped over the lazy dog", 
                max_length = 6).chunk()

Converted --> The quick brown fox jumped over the lazy dog
into --> [['The', 'quick', 'brown', 'fox', 'jumped', 'over'], ['the', 'lazy', 'dog']]


[['The', 'quick', 'brown', 'fox', 'jumped', 'over'], ['the', 'lazy', 'dog']]

***

This next example demonstrates providing a user the ability to chunk and store if they want to. By providing a name with no file path as db_target_name, it will create a json file local to wherever you run this jupyter notebook. If you were to provide a full file path e.g C:\My Documents\LocalStorageExample.json it would store it away from the jupyter notebook directory

In [14]:
SentenceChunker(sentence = "The quick brown fox jumped over the lazy dog", 
                max_length = 6, 
                db_target_name = "LocalStorageExample.json").chunk_and_store()

Converted --> The quick brown fox jumped over the lazy dog
into --> [['The', 'quick', 'brown', 'fox', 'jumped', 'over'], ['the', 'lazy', 'dog']]
Inserted into database!


It may be useful to prove that it has actually been added into the database, the below code is one way you could quickly check that:

In [15]:
TinyDB("LocalStorageExample.json").all()

[{'sentence': 'The quick brown fox jumped over the lazy dog',
  'max_length': 6,
  'chunk_results': [['The', 'quick', 'brown', 'fox', 'jumped', 'over'],
   ['the', 'lazy', 'dog']]},
 {'sentence': 'The quick brown fox jumped over the lazy dog',
  'max_length': 6,
  'chunk_results': [['The', 'quick', 'brown', 'fox', 'jumped', 'over'],
   ['the', 'lazy', 'dog']]},
 {'sentence': 'The quick brown fox jumped over the lazy dog',
  'max_length': 6,
  'chunk_results': [['The', 'quick', 'brown', 'fox', 'jumped', 'over'],
   ['the', 'lazy', 'dog']]}]

It's possible to be more involved than this and search for the criteria you've just added into the db and ensure that it's been succesfully added - but for the sake of test, a visual assertion is okay. 

## Q3 - Please write a code review...

For the following function, highlighting both positive and negative aspects: 

---

Note/Plan: 
* As I don't wish to reveal the code within my github, I will write the review here in expectation that the company will refer back to their original PDF document.

***

### Positive Aspects: 
* The name of the function is clear
* Makes good use of objects to carry attributes across

### Negative Aspects:
* The function could do with a doc string defining in further detail what's happening within the function
* The use of self suggests that this is used within a Class, would want to see the usage of this function in the wider example to understand if this is the best location or simply gain further context. 
* The use of "shortcuts" within the nested if statement although timesaving, impacts readability. e.g "if class_assignment:" is shorthand for "if clan_assignment = True" 
* The combination of shortcut if statement, with comment next door could be combined into a more detailed if statement. This would improve readability, and remove a needless comment. e.g "if clan_assignment: # Check if a player is in a clan" -> "if clan_assignment = True" 
* Should the function create clan contain a delete or remove aspect within itself? Should that be extracted out into it's own logic in order to better isolate the "create clan" function? 
* The return object seems unclear in relation to "create clan", 

### Can't decide aspects: 
* I've always known the use of the unpacking arguments trick ** to be somewhat frowned upon as it can impact readability. At the line self.repo.create_clean(**data) would it be worth considering activating the method with those properties as opposed to creating a dict called data and then unpacking it?
* I would want to isolate the individual aspects of clan_assignment check, clan_assignment_tidy_up, clan_request_purge, create_new_clan_on_system, and add_as_owner as seperate methods, and use "create_clan" as a wrapper
* Should there be a dup name check on the clan name?  



## Q4 - Outline the unit tests...

Outline the unit tests you would write for the function listed in Q3:

1. Is it possible to succesfully return the clan assignment for the current user? 
    * Can you prove that the user you've requested and the result are correct? You haven't picked up someone else? 
2. Can you confirm it's possible to actually remove player from current clan? 
    * Can you take a snapshot of a clan member list?
    * "mock" the clan (so you dont actually remove them whilst testing but can mimic all the same attribute)
    * Remove the player from the "mocked clan"
    * Prove there's a difference between the clan member list at the start and the mocked clan member list? The difference being the user you wished to remove? 
3. Can you delete all open clan request for players succesfully?
    * Get the player object 
    * Mock the player object
    * Delete requests from player object
    * Assert player_object.clan_requests != mock_player.clan_requests 
    * Assert len(mock_player.clan_requests) == 0
4. Is a new clan successfully created on request?
    * Pass the arguments necessary to create a new clan 
    * Mock the db element? 
    * Can you find the details of the new clan present within the db?
5. Is the player succesfully added as the owner on request?
    * Locate the clan object, 
    * Can you mock adding them to the clan as the owner? 
    * Can you now find that the player == owner? 
    * What happens if someone else is added just before? - How does the error handling hold up? 
6. Can you update clan data to get correct member count? 

## Q5 - Please turn the following requirements...

Please turn the following requirements into 5 Acceptance Criteria of your choice. As the requirement is quite broad, feel free to choose what features your implementation will support. 

*We need a software capable of managing an automated coffee vending machine. The machine must be able to take an order between different variants of coffee preparations, take the money associated with the product, prepare the beverage and highlight a number of error events*.

***

#### Assumptions Made

Assuming different variants of coffee preparations == different types of coffee as as opposed to different stages of preparing coffee for several customers; I'm picturing a coffee machine in a petrol station as an example, handling one user at a time

Assuming that the elements listed in the initial statement are the features, I've decided to implement the below: 

*** 

The machine must be able to take an order between different variants of coffee preparations

***

1. Starting to take the order 

    * Given a customer has requested a coffee
    * And the machine is available to take an order 
    * When the customer presses "coffee" button
    * Then the software should access its database of coffee variants
    * And present a selection of coffee variants for the customer to order from



2. Recognising the order

    * Given the customer has selected a coffee from the presented selection
    * And the machine is available to take an order 
    * When the customer selects the variant
    * Then the software should recognise the customer input
    * And the sofware should accept and store that customer input



3. Confirming with the user prior to accepting payment

    * Given the software has accepted the customer input
    * And the customer is ready to make payment for the order 
    * When the software accepts and stores the order
    * Then the software should confirm with the customer that the stored details are correct
    * And present to the customer an opportunity to confirm their order



4. The software is able to accept user confirmation of their order and handle accordingly

    * Positive: 
        * Given the customer has been asked to confirm their order
        * And the software is waiting for a response
        * When the customer confirms their order is correct 
        * Then the software is able to store the customer order as a confirmed order
        * And the software is able to move on to take secure payment for the order
    * Negative: 
        * Given the customer has been asked to confirm their order
        * And the software is waiting for a response
        * When the customer confirms their order is incorrect 
        * Then the software is able to place the customer at the beginning of the process to start a new order
        * And the software is able to clear out "previous information" from the customer


5. The software is able to move on to the money handling process after confirming and taking the order

    * Given the software has accepted and stored the confirmed order 
    * And the customer is ready to make payment for the order
    * When the software accepts the positive confirmation from the customer
    * Then the software should proceed to the money handling process
    * And prepare to receive payment for the order



## Q6 - Please draw a high-level diagram (in a tool of your choice)...

Outlining a possible implementation for the Acceptance Criteria identified in Q5

Please refer to the image stored in the repo as "HighLevelDiagram"
